In [30]:
import json
import datetime
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import openpyxl
import math
today_foloder="20180416" # str(datetime.datetime.now().year)+str(datetime.datetime.now().month).zfill(2)+str(datetime.datetime.now().day).zfill(2)
sales_file_date="2018-04-07"

json_list_date=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/*.json")

start = datetime.datetime(2018,2,11).date()
end = datetime.datetime(2018,4,7).date()
weather_date_list = [start + datetime.timedelta(days=x) for x in range(0, (end-start+datetime.timedelta(days=1)).days)]


In [31]:
sales=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_"+sales_file_date+"/wide_sales_date"+sales_file_date+".xlsx",sheetname="sales")

In [32]:
store_zip=pd.read_excel("/home/jian/Projects/Big_Lots/Other Input/all_store_DMA.xlsx",dtype=str)
store_zip=store_zip[store_zip['zip_cd']!='nan']
store_zip=store_zip[['location_id','zip_cd']]
store_zip=store_zip.drop_duplicates().reset_index()
del store_zip['index']
store_zip['location_id']=store_zip['location_id'].astype(int)

In [33]:
Week_End_Date_List=[datetime.datetime(2018,2,17).date(),datetime.datetime(2018,2,24).date(),datetime.datetime(2018,3,3).date(),
                    datetime.datetime(2018,3,10).date(),datetime.datetime(2018,3,17).date(),datetime.datetime(2018,3,24).date(),
                    datetime.datetime(2018,3,31).date(),datetime.datetime(2018,4,7).date()]

In [34]:
sales_Q1=store_zip
for date in Week_End_Date_List:
    sales_week=sales[['location_id',str(date),str(date+datetime.timedelta(days=-364))]]
    sales_week=sales_week[(sales[str(date)]>0) & (sales_week[str(date+datetime.timedelta(days=-364))]>0)]
    sales_week.reset_index(inplace=True)
    del sales_week['index']
    sales_week.columns=[['location_id','Sales_2018_'+str(date),'Sales_2017_'+str(date)]]
    sales_week['YoY_Dollar_'+str(date)]=sales_week['Sales_2018_'+str(date)]-sales_week['Sales_2017_'+str(date)]
    sales_week['YoY_Pctg_'+str(date)]=sales_week['YoY_Dollar_'+str(date)]/sales_week['Sales_2017_'+str(date)]
    sales_Q1=pd.merge(sales_Q1,sales_week,on="location_id",how="inner")


In [35]:
sales_Q1_Dollar_Columns=["YoY_Dollar_"+str(date) for date in Week_End_Date_List]
sales_Q1_Dollar=sales_Q1[["location_id","zip_cd"]+sales_Q1_Dollar_Columns]

sales_Q1_Pctg_Columns=["YoY_Pctg_"+str(date) for date in Week_End_Date_List]
sales_Q1_Pctg=sales_Q1[["location_id","zip_cd"]+sales_Q1_Pctg_Columns]

unique_zip=sales_Q1['zip_cd'].unique()

In [36]:
remove_date=[]
for date in weather_date_list:
    try:
        date_str=str(date)
        response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))
    except:
        print(date)
        remove_date=remove_date+[date]
weather_date_list.remove(datetime.datetime(2018,2,27).date())
weather_date_list.remove(datetime.datetime(2018,3,23).date())
# Remove the following 2 dates from the json weather list

2018-02-27
2018-03-23


In [10]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/Q1_Weeks/BL_Weather_8weeks_5ivs_YoY_Dollar.xlsx",engine='xlsxwriter')

In [24]:
weather_groups=['Clear','Snow','Rain']
storage={}
for weather in weather_groups:
    df_sales=sales_Q1_Dollar.copy()


    for date in weather_date_list:
        df=pd.DataFrame(index=unique_zip)
        df.index.name = 'zip_cd'
        date_str=str(date)
        df[weather+"_"+date_str]=0
        response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))

        for zip_cd in list(response.keys()):
            if zip_cd in unique_zip:
                for k in range(len(response[zip_cd]['weather'])):
                    if response[zip_cd]['weather'][k]['main']==weather:
                        df[weather+"_"+date_str][zip_cd]=1
        df.reset_index(inplace=True)
        
        df_sales=pd.merge(df_sales,df,on="zip_cd",how="left")

    df=df_sales
    
    df_columns=["location_id","zip_cd"]
    for week_end_date in Week_End_Date_List:
        week_list=[]
        for i in range(7):
            weather_date=week_end_date-datetime.timedelta(days=(6-i))
            if weather_date in weather_date_list:
                col=weather+"_"+str(weather_date)
                week_list=week_list+[col]
        week_list=week_list+["YoY_Dollar_"+str(week_end_date)]
        df_columns=df_columns+week_list
    df=df[df_columns]
    storage.update({weather:df})
    df.to_excel(writer,weather,index=False)
    print(weather)

Clear
Snow
Rain


In [12]:
df_temp=sales_Q1_Dollar.copy()
for date in weather_date_list:
    date_str=str(date)
    df=pd.DataFrame(index=unique_zip)
    df.index.name = 'zip_cd'
    
    df["Temp_max_"+date_str]=np.nan
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))
    for zip_cd in list(response.keys()):
        if zip_cd in unique_zip:
            temp_max=response[zip_cd]['main']['temp_max'] * 9/5 - 459.67
            df["Temp_max_"+date_str][zip_cd]=temp_max
    df.reset_index(inplace=True)

    df_temp=pd.merge(df_temp,df,on="zip_cd",how="left")

df_columns=["location_id","zip_cd"]
for week_end_date in Week_End_Date_List:
    week_list=[]
    for i in range(7):
        weather_date=week_end_date-datetime.timedelta(days=(6-i))
        if weather_date in weather_date_list:
            col="Temp_max_"+str(weather_date)
            week_list=week_list+[col]
    week_list=week_list+["YoY_Dollar_"+str(week_end_date)]
    df_columns=df_columns+week_list
df_columns=[col.rstrip('\r\n') for col in df_columns]
df_temp=df_temp[df_columns]
storage.update({"Temp_max":df_temp})
df_temp.to_excel(writer,"Temp_max",index=False)


In [13]:
df_wind=sales_Q1_Dollar.copy()
for date in weather_date_list:
    date_str=str(date)
    df=pd.DataFrame(index=unique_zip)
    df.index.name = 'zip_cd'
    
    df["Wind_speed_"+date_str]=np.nan
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))
    for zip_cd in list(response.keys()):
        if zip_cd in unique_zip:
            wind_speed=response[zip_cd]['wind']['speed']
            df["Wind_speed_"+date_str][zip_cd]=float(wind_speed)
    df.reset_index(inplace=True)

    df_wind=pd.merge(df_wind,df,on="zip_cd",how="left")

df_columns=["location_id","zip_cd"]
for week_end_date in Week_End_Date_List:
    week_list=[]
    for i in range(7):
        weather_date=week_end_date-datetime.timedelta(days=(6-i))
        if weather_date in weather_date_list:
            col="Wind_speed_"+str(weather_date)
            week_list=week_list+[col]
    week_list=week_list+["YoY_Dollar_"+str(week_end_date)]
    df_columns=df_columns+week_list

df_wind=df_wind[df_columns]
storage.update({"Wind_speed":df_wind})
df_wind.to_excel(writer,"Wind_speed",index=False)


In [14]:
writer.save()

In [15]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/Q1_Weeks/BL_Weather_8weeks_5ivs_YoY_Pctg.xlsx",engine='xlsxwriter')

In [16]:
sales_Q1_Pctg.head(2)

,location_id,zip_cd,YoY_Pctg_2018-02-17,YoY_Pctg_2018-02-24,YoY_Pctg_2018-03-03,YoY_Pctg_2018-03-10,YoY_Pctg_2018-03-17,YoY_Pctg_2018-03-24,YoY_Pctg_2018-03-31,YoY_Pctg_2018-04-07
0,1493,01060,0.178844,-0.139636,-0.007172,0.123759,0.124528,0.139659,0.275595,-0.104765
1,1863,01089,0.050643,-0.253947,-0.169881,0.129053,0.089379,0.046839,-0.001417,-0.152419


In [17]:
weather_groups=['Clear','Snow','Rain']
storage={}
for weather in weather_groups:
    df_sales=sales_Q1_Pctg.copy()


    for date in weather_date_list:
        df=pd.DataFrame(index=unique_zip)
        df.index.name = 'zip_cd'
        date_str=str(date)
        df[weather+"_"+date_str]=0
        response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))

        for zip_cd in list(response.keys()):
            if zip_cd in unique_zip:
                for k in range(len(response[zip_cd]['weather'])):
                    if response[zip_cd]['weather'][k]['main']==weather:
                        df[weather+"_"+date_str][zip_cd]=1
        df.reset_index(inplace=True)
        
        df_sales=pd.merge(df_sales,df,on="zip_cd",how="left")

    df=df_sales
    
    df_columns=["location_id","zip_cd"]
    for week_end_date in Week_End_Date_List:
        week_list=[]
        for i in range(7):
            weather_date=week_end_date-datetime.timedelta(days=(6-i))
            if weather_date in weather_date_list:
                col=weather+"_"+str(weather_date)
                week_list=week_list+[col]
        week_list=week_list+["YoY_Pctg_"+str(week_end_date)]
        df_columns=df_columns+week_list
    df=df[df_columns]
    storage.update({weather:df})
    df.to_excel(writer,weather,index=False)
    print(weather)

Clear
Snow
Rain


In [18]:
df_temp=sales_Q1_Pctg.copy()
for date in weather_date_list:
    date_str=str(date)
    df=pd.DataFrame(index=unique_zip)
    df.index.name = 'zip_cd'
    
    df["Temp_max_"+date_str]=np.nan
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))
    for zip_cd in list(response.keys()):
        if zip_cd in unique_zip:
            temp_max=response[zip_cd]['main']['temp_max'] * 9/5 - 459.67
            df["Temp_max_"+date_str][zip_cd]=temp_max
    df.reset_index(inplace=True)

    df_temp=pd.merge(df_temp,df,on="zip_cd",how="left")

df_columns=["location_id","zip_cd"]
for week_end_date in Week_End_Date_List:
    week_list=[]
    for i in range(7):
        weather_date=week_end_date-datetime.timedelta(days=(6-i))
        if weather_date in weather_date_list:
            col="Temp_max_"+str(weather_date)
            week_list=week_list+[col]
    week_list=week_list+["YoY_Pctg_"+str(week_end_date)]
    df_columns=df_columns+week_list
df_columns=[col.rstrip('\r\n') for col in df_columns]
df_temp=df_temp[df_columns]
storage.update({"Temp_max":df_temp})
df_temp.to_excel(writer,"Temp_max",index=False)


In [19]:
df_wind=sales_Q1_Pctg.copy()
for date in weather_date_list:
    date_str=str(date)
    df=pd.DataFrame(index=unique_zip)
    df.index.name = 'zip_cd'
    
    df["Wind_speed_"+date_str]=np.nan
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+date_str+".json",'r'))
    for zip_cd in list(response.keys()):
        if zip_cd in unique_zip:
            wind_speed=response[zip_cd]['wind']['speed']
            df["Wind_speed_"+date_str][zip_cd]=float(wind_speed)
    df.reset_index(inplace=True)

    df_wind=pd.merge(df_wind,df,on="zip_cd",how="left")

df_columns=["location_id","zip_cd"]
for week_end_date in Week_End_Date_List:
    week_list=[]
    for i in range(7):
        weather_date=week_end_date-datetime.timedelta(days=(6-i))
        if weather_date in weather_date_list:
            col="Wind_speed_"+str(weather_date)
            week_list=week_list+[col]
    week_list=week_list+["YoY_Pctg_"+str(week_end_date)]
    df_columns=df_columns+week_list

df_wind=df_wind[df_columns]
storage.update({"Wind_speed":df_wind})
df_wind.to_excel(writer,"Wind_speed",index=False)


In [20]:
writer.save()

In [37]:
response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+"2018-02-11"+".json",'r'))